In [ ]:
from dputils.scrape import Scraper,Tag

In [ ]:
def get_url_from_amazon(url):
    sc=Scraper(url)
    urlList=sc.get_repeating_date_from_page(
        target=Tag('body'),                      #page div tag where we can find it complete block by inspecting
        items=Tag('div',cls="s-result-item"),    #Perticular detailed page  div tag where we can find it complete block by inspecting
        url=Tag('a',cls="a-link-normal",output="href"),   #Link for opening the perticular page
    )
    return [item.get('url') for item in urlList if item.get('url')]

In [ ]:
url="https://www.amazon.in/s?k=mobile&rh=n%3A1389401031&ref=nb_sb_noss"
get_url_from_amazon(url)

# Database Setup

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker,declarative_base
from sqlalchemy import Column,Integer,String,Boolean
Base=declarative_base()
class Products(Base):
    __tablename__='products'
    id=Column(Integer,primary_key=True)
    url=Column(String,unique=True)
    title=Column(String)
    sprice=Column(String)
    oprice=Column(String)
    reviews=Column(String)

engine=create_engine('sqlite:///amazon.db')
Base.metadata.create_all(engine)

def opendb():
    engine = create_engine('sqlite:///amazon.db')
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

In [ ]:
def get_details_from_url(product_url):
    if 'html' in product_url:
        return None
    if product_url.startswith('/'):
        base_url="https://www.amazon.in"
        product_url = base_url + product_url
    sc=Scraper(product_url)
    product=sc.get_data_from_page(
        name=Tag('h1',id='title'),
        sprice=Tag('span',cls="priceToPay"),
        oprice=Tag('span',cls="basisPrice"),
        reviews=Tag('span',id="acrCustomerReviewText")
    )
    return product

In [ ]:
for product_url in get_url_from_amazon(url):
    product=get_details_from_url(product_url)
    if not product:
        continue
    if not product['name']:
        continue
    db=opendb()
    item=Products(
        url=product_url,
        title = product['name'].strip(),
        sprice = product['sprice'].strip(),
        oprice = product['oprice'].strip(),
        reviews = product['reviews'].strip(),
    )
    try:
        db.add(item)
        db.commit()
    except:
        print("Product details already in database")
    finally:
        db.close()
    print("Product details added to database")